In [1]:
import os
import re
import subprocess
import pandas as pd
from bs4 import BeautifulSoup, Tag

In [3]:
# imported from Zach's computer, where he (ChatGPT API) wrapped the java functions in class headers
filedir = os.listdir("wrapped_functions")

In [3]:
### ONLY NEED TO RUN THIS ONCE TO PARSE

# # parsing each function with srcml
# for file in filedir:
#     infile = f"wrapped_functions/{file}"
#     outfile = f"{infile[:-5]}.xml"

#     subprocess.run(["srcml", infile, "--position", "-o", outfile])

CompletedProcess(args=['srcml', 'wrapped_functions/genSql_wrapped.java', '--position', '-o', 'wrapped_functions/genSql_wrapped.xml'], returncode=0)

In [3]:
#infile = f"wrapped_functions/getBackCommand12_wrapped.java"
#infile = f"wrapped_functions/getImageWithSource_wrapped.java"
#infile = "wrapped_functions/updateSchema_wrapped.java" 
infile = "refreshTreePanel.java"
outfile = f"{infile[:-5]}.xml"

subprocess.run(["srcml", infile, "--position", "-o", outfile])


CompletedProcess(args=['srcml', 'refreshTreePanel.java', '--position', '-o', 'refreshTreePanel.xml'], returncode=0)

In [2]:
pointless = {',':0,     '(':0,   ')':0,   '()':0, '{':0,  '}':0,   ');':0, 
             '),':0,    '));':0, '];':0,  '[':0,  ']':0,  '))':0,  '){':0,  
             ';':0,     ');':0,  '});':0, '};':0, '((':0, ')){':0, ')[':0, 
             '))));':0, ')))':0, ')(':0,  '.':0}


def print_tags(root, tag, stack=None):
    global annotations
    if not stack:
        stack = []
        
    if isinstance(tag, Tag):
        stack.append(tag.name)
        for child in tag.children:
            print_tags(root, child, stack)
        stack.pop()
        
    elif tag: 
        tag = tag.strip()
        if tag and tag not in pointless:
            if tag not in occurrences:
                occurrences[tag] = 0
            else:
                occurrences[tag] += 1

            annotations.append({'word': tag, 'occurrence': occurrences[tag],
                   'tree_parts': stack.copy()})

In [3]:
#with open('./xml_trees/liesBetween_wrapped.xml', 'r') as f:
#with open('./xml_trees/markStart_wrapped.xml', 'r') as f:   
#trees = os.listdir('xml_trees/')
#trees = ['refreshTreePanel.xml']
trees = ['pluginRegistryContains_wrapped.xml']
# trees = ['getBackCommand12_wrapped.xml']
# trees = ['getImageWithSource_wrapped.xml']
# trees = ['getRelFeedItems_wrapped.java', 'getStatusFlagFields_wrapped.java', 'removeSer']
for trunk in trees: 
    annotations = []
    occurrences = {}
    name = re.sub("_wrapped.xml", "", trunk)
    
    with open(f'xml_trees/{trunk}', 'r') as f:
        data = f.read()

    soup = BeautifulSoup(data, features="xml")
    soup = soup.find('function')

    print_tags(root=soup, tag=soup)
    annotations = pd.DataFrame(annotations)

    annotations.to_csv(f'annotations/{name}_annotated.csv', index=False)